In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Set visualisation styles
sns.set(style="whitegrid")

# Load data
base_path = 'D:\\SMU_MLE_Apr2025\\Assignment 1\\data'  # Adjust this path if needed

features_attributes = pd.read_csv(os.path.join(base_path, 'features_attributes.csv'))
lms_loan_daily = pd.read_csv(os.path.join(base_path, 'lms_loan_daily.csv'))
features_financials = pd.read_csv(os.path.join(base_path, 'features_financials.csv'))
features_clickstream = pd.read_csv(os.path.join(base_path, 'feature_clickstream.csv'))

In [3]:
# Helper function for initial data check
def explore_df(df, name):
    print(f"\n--- {name} ---")
    print("Shape:", df.shape)
    print("\nColumns:", df.columns.tolist())
    print("\nData Types:\n", df.dtypes)
    print("\nMissing Values:\n", df.isnull().sum())
    print("\nSample Data:\n", df.head())

In [4]:
# Explore each dataset
explore_df(features_attributes, "features_attributes")
explore_df(lms_loan_daily, "lms_loan_daily")
explore_df(features_financials, "features_financials")
explore_df(features_clickstream, "features_clickstream")


--- features_attributes ---
Shape: (12500, 6)

Columns: ['Customer_ID', 'Name', 'Age', 'SSN', 'Occupation', 'snapshot_date']

Data Types:
 Customer_ID      object
Name             object
Age              object
SSN              object
Occupation       object
snapshot_date    object
dtype: object

Missing Values:
 Customer_ID      0
Name             0
Age              0
SSN              0
Occupation       0
snapshot_date    0
dtype: int64

Sample Data:
   Customer_ID            Name Age          SSN     Occupation snapshot_date
0  CUS_0x1000  Alistair Barrf  18  913-74-1218         Lawyer    01-05-2023
1  CUS_0x1009          Arunah  26  063-67-6938       Mechanic    01-01-2025
2  CUS_0x100b        Shirboni  19    #F%$D@*&8  Media_Manager    01-03-2024
3  CUS_0x1011       Schneyerh  44  793-05-8223         Doctor    01-11-2023
4  CUS_0x1013        Cameront  44  930-49-9615       Mechanic    01-12-2023

--- lms_loan_daily ---
Shape: (137500, 11)

Columns: ['loan_id', 'Customer_ID', 'loan

In [5]:
# Basic summary statistics
print("\n--- Summary Statistics ---")
print("\nfeatures_attributes:\n", features_attributes.describe(include='all'))
print("\nlms_loan_daily:\n", lms_loan_daily.describe(include='all'))
print("\nfeatures_financials:\n", features_financials.describe(include='all'))
print("\nfeatures_clickstream:\n", features_clickstream.describe())


--- Summary Statistics ---

features_attributes:
        Customer_ID      Name    Age        SSN Occupation snapshot_date
count        12500     12500  12500      12500      12500         12500
unique       12500     10139    301      11798         16            25
top     CUS_0x1000  Jessicad     26  #F%$D@*&8    _______    01-08-2024
freq             1         6    364        703        880           543

lms_loan_daily:
                       loan_id Customer_ID loan_start_date    tenure  \
count                  137500      137500          137500  137500.0   
unique                  12500       12500              25       NaN   
top     CUS_0x1000_2023_05_01  CUS_0x1000      01-08-2024       NaN   
freq                       11          11            5973       NaN   
mean                      NaN         NaN             NaN      10.0   
std                       NaN         NaN             NaN       0.0   
min                       NaN         NaN             NaN      10.0   
25%

In [6]:
# Date parsing and checks
for df, label in [
    (features_attributes, 'features_attributes'),
    (lms_loan_daily, 'lms_loan_daily'),
    (features_financials, 'features_financials'),
    (features_clickstream, 'features_clickstream')
]:
    if 'snapshot_date' in df.columns:
        df['snapshot_date'] = pd.to_datetime(df['snapshot_date'], errors='coerce')
        print(f"{label} date range: {df['snapshot_date'].min()} to {df['snapshot_date'].max()}")

features_attributes date range: 2023-01-01 00:00:00 to 2025-01-01 00:00:00
lms_loan_daily date range: 2023-01-01 00:00:00 to 2025-01-11 00:00:00
features_financials date range: 2023-01-01 00:00:00 to 2025-01-01 00:00:00
features_clickstream date range: 2023-01-01 00:00:00 to 2024-12-01 00:00:00


In [7]:
# Remove characters from "_" onwards in Age column
features_attributes["Age"] = features_attributes["Age"].astype(str).str.replace(r'_.*', '', regex=True)

# Convert to numeric (will still catch things like "abc" as NaN)
features_attributes["Age"] = pd.to_numeric(features_attributes["Age"], errors="coerce")

# Check if age is valid
features_attributes["is_age_valid"] = features_attributes["Age"].between(18, 99)

# SSN validity check using regex
features_attributes["is_ssn_valid"] = features_attributes["SSN"].str.match(r'^\d{3}-\d{2}-\d{4}$')


In [8]:
features_attributes.head(20)

,Customer_ID,Name,Age,SSN,Occupation,snapshot_date,is_age_valid,is_ssn_valid
0,CUS_0x1000,Alistair Barrf,18,913-74-1218,Lawyer,2023-01-05,True,True
1,CUS_0x1009,Arunah,26,063-67-6938,Mechanic,2025-01-01,True,True
2,CUS_0x100b,Shirboni,19,#F%$D@*&8,Media_Manager,2024-01-03,True,False
3,CUS_0x1011,Schneyerh,44,793-05-8223,Doctor,2023-01-11,True,True
4,CUS_0x1013,Cameront,44,930-49-9615,Mechanic,2023-01-12,True,True
5,CUS_0x1015,Holtono,27,810-97-7024,Journalist,2023-01-08,True,True
6,CUS_0x1018,Felsenthalq,15,731-19-8119,Accountant,2023-01-11,False,True
7,CUS_0x1026,Josephv,52,500-62-9044,Manager,2023-01-10,True,True
8,CUS_0x102d,Neil Chatterjeex,31,692-71-7552,Entrepreneur,2024-01-01,True,True
9,CUS_0x102e,Rhysn,26,#F%$D@*&8,Scientist,2024-01-04,True,False


In [9]:
print(features_attributes.value_counts("is_age_valid"))
print(features_attributes.value_counts("is_ssn_valid"))
print(features_attributes.isnull().sum().sum())

is_age_valid
True     11512
False      988
Name: count, dtype: int64
is_ssn_valid
True     11797
False      703
Name: count, dtype: int64
0
